## 숙제 1

In [82]:
! pip install torch-tensorrt

In [83]:
pip install git+https://github.com/NVIDIA/TensorRT-Model-Optimizer.git

  Cloning https://github.com/NVIDIA/TensorRT-Model-Optimizer.git to /tmp/pip-req-build-373vdsfj
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/TensorRT-Model-Optimizer.git /tmp/pip-req-build-373vdsfj
  Resolved https://github.com/NVIDIA/TensorRT-Model-Optimizer.git to commit 1574baaab3f3e34ffab2fa25439dd3f019aab512
ERROR: git+https://github.com/NVIDIA/TensorRT-Model-Optimizer.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [84]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch_tensorrt

### 1. 데이터 로드 및 전처리

In [85]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


### 2. Alexnet 모델 정의 (PyTorch 사용)

In [86]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features  = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.AdaptiveAvgPool2d((6, 6))
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

### 3. 모델 학습 설정

In [87]:
# 3. 모델 학습 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AlexNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 4. 모델 학습

In [88]:
num_epochs = 20
patience = 2  # Early stopping patience
best_loss = float('inf')
stopping_step = 0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        # 기존의 기울기 초기화
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 손실 값 출력
        running_loss += loss.item()
        if i % 100 == 99:    # 매 100 미니배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

        # Early stopping 조건 확인
    if running_loss < best_loss:
        best_loss = running_loss
        stopping_step = 0
    else:
        stopping_step += 1
        if stopping_step >= patience:
            print('Early stopping triggered')
            break

print('학습 완료')

[Epoch 1, Batch 100] loss: 2.020
[Epoch 1, Batch 200] loss: 1.657
[Epoch 1, Batch 300] loss: 1.468
[Epoch 2, Batch 100] loss: 1.229
[Epoch 2, Batch 200] loss: 1.171
[Epoch 2, Batch 300] loss: 1.118
[Epoch 3, Batch 100] loss: 0.983
[Epoch 3, Batch 200] loss: 0.950
[Epoch 3, Batch 300] loss: 0.916
[Epoch 4, Batch 100] loss: 0.792
[Epoch 4, Batch 200] loss: 0.790
[Epoch 4, Batch 300] loss: 0.793
[Epoch 5, Batch 100] loss: 0.692
[Epoch 5, Batch 200] loss: 0.694
[Epoch 5, Batch 300] loss: 0.687
[Epoch 6, Batch 100] loss: 0.594
[Epoch 6, Batch 200] loss: 0.626
[Epoch 6, Batch 300] loss: 0.614
[Epoch 7, Batch 100] loss: 0.514
[Epoch 7, Batch 200] loss: 0.539
[Epoch 7, Batch 300] loss: 0.538
[Epoch 8, Batch 100] loss: 0.429
[Epoch 8, Batch 200] loss: 0.487
[Epoch 8, Batch 300] loss: 0.499
[Epoch 9, Batch 100] loss: 0.410
[Epoch 9, Batch 200] loss: 0.428
[Epoch 9, Batch 300] loss: 0.433
[Epoch 10, Batch 100] loss: 0.346
[Epoch 10, Batch 200] loss: 0.386
[Epoch 10, Batch 300] loss: 0.406
[Epoch 

### 5. 모델 평가

In [89]:
# 5. 테스트 정확도 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'최종 테스트 정확도: {100 * correct / total:.4f}%')

최종 테스트 정확도: 77.1100%


## 숙제2 (이어서)

### 6. Torch-TensorRT 적용

In [90]:
# 모델 평가 모드 설정
model.eval().to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): AdaptiveAvgPool2d(output_size=(6, 6))
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
   

In [91]:
# Torch-TensorRT를 사용해 모델 최적화
inputs = [torch.rand((1, 3, 32, 32)).to(device)]
enabled_precisions = {torch.float}
debug = True
workspace_size = 20 << 30
min_block_size = 7
torch_executed_ops = {}

optimized_model = torch_tensorrt.compile(
    model,
    ir="torch_compile",
    inputs=inputs,
    enabled_precisions=enabled_precisions,
    debug=debug,
    workspace_size=workspace_size,
    min_block_size=min_block_size,
    torch_executed_ops=torch_executed_ops,
)

In [92]:
# Inference 속도 비교
import time

def measure_inference_time(model, inputs, num_tests=100):
    start_time = time.time()
    with torch.no_grad():
        for _ in range(num_tests):
            try:
                _ = model(*inputs)
            except Exception as e:
                print(f"Error during inference: {e}")
                break
    end_time = time.time()
    return (end_time - start_time) / num_tests

In [93]:
# 기본 모델 Inference 시간 측정
model.eval()
original_time = measure_inference_time(model, inputs)
print(f'기존 모델 평균 Inference 시간: {original_time:.6f} 초')

기존 모델 평균 Inference 시간: 0.001826 초


In [94]:
# Torch-TensorRT 최적화 모델 Inference 시간 측정
optimized_time = measure_inference_time(optimized_model, inputs)
print(f'Torch-TensorRT 최적화 모델 평균 Inference 시간: {optimized_time:.6f} 초')

INFO:torch_tensorrt.dynamo.utils:Using Default Torch-TRT Runtime (as requested by user)
INFO:torch_tensorrt.dynamo.utils:Device not specified, using Torch default current device - cuda:0. If this is incorrect, please specify an input device, via the device keyword.
INFO:torch_tensorrt.dynamo.utils:Compilation Settings: CompilationSettings(enabled_precisions={<dtype.f16: 6>}, debug=True, workspace_size=21474836480, min_block_size=7, torch_executed_ops={}, pass_through_build_failures=False, max_aux_streams=None, version_compatible=False, optimization_level=None, use_python_runtime=False, truncate_double=False, use_fast_partitioner=True, enable_experimental_decompositions=False, device=Device(type=DeviceType.GPU, gpu_id=0), require_full_compilation=False, disable_tf32=False, assume_dynamic_shape_support=False, sparse_weights=False, make_refittable=False, engine_capability=<EngineCapability.STANDARD: 1>, num_avg_timing_iters=1, dla_sram_size=1048576, dla_local_dram_size=1073741824, dla_glo

Error during inference: backend='torch_tensorrt_backend' raised:
ValueError: __len__() should return >= 0

While executing %_adaptive_avg_pool2d : [num_users=1] = call_function[target=torch.ops.aten._adaptive_avg_pool2d.default](args = (%max_pool2d_default_2, [6, 6]), kwargs = {_itensor_to_tensor_meta: {<tensorrt_bindings.tensorrt.ITensor object at 0x7cc010b9a630>: ((1, 3, 32, 32), torch.float32, False, (3072, 1024, 32, 1), torch.contiguous_format, False, {}), <tensorrt_bindings.tensorrt.ITensor object at 0x7cc010a85670>: ((64, 3, 3, 3), torch.float32, True, (27, 9, 3, 1), torch.contiguous_format, False, {}), <tensorrt_bindings.tensorrt.ITensor object at 0x7cc010a96230>: ((64,), torch.float32, True, (1,), torch.contiguous_format, False, {}), <tensorrt_bindings.tensorrt.ITensor object at 0x7cc010b6ae70>: ((1, 64, 32, 32), torch.float32, False, (65536, 1024, 32, 1), torch.contiguous_format, False, {}), <tensorrt_bindings.tensorrt.ITensor object at 0x7cc010b402f0>: ((1, 64, 32, 32), torch

In [95]:
# 최종 결과 출력
speedup = original_time / optimized_time
print(f'Torch-TensorRT 적용으로 인한 Inference 속도 향상: {speedup:.2f}배')

Torch-TensorRT 적용으로 인한 Inference 속도 향상: 0.07배


In [96]:
# Torch 유틸리티로 작업 공간 정리
torch._dynamo.reset()

### 7. PyTorch JIT Trace 모드 적용

In [97]:
# 모델을 JIT Trace 모드로 변환
traced_model = torch.jit.trace(model, torch.rand((1, 3, 32, 32)).to(device))

In [98]:
# JIT Trace 모델 Inference 시간 측정
jit_time = measure_inference_time(traced_model, inputs)
print(f'JIT Trace 모델 평균 Inference 시간: {jit_time:.6f} 초')

JIT Trace 모델 평균 Inference 시간: 0.001088 초


In [99]:
# 최종 결과 출력
jit_speedup = original_time / jit_time
print(f'JIT Trace 적용으로 인한 Inference 속도 향상: {jit_speedup:.2f}배')

JIT Trace 적용으로 인한 Inference 속도 향상: 1.68배


In [100]:
# JIT Trace 모델 그래프 및 코드 출력
print(traced_model.graph)
print(traced_model.code)

graph(%self.1 : __torch__.___torch_mangle_290.AlexNet,
      %x : Float(1, 3, 32, 32, strides=[3072, 1024, 32, 1], requires_grad=0, device=cuda:0)):
  %classifier : __torch__.torch.nn.modules.container.___torch_mangle_289.Sequential = prim::GetAttr[name="classifier"](%self.1)
  %features : __torch__.torch.nn.modules.container.___torch_mangle_280.Sequential = prim::GetAttr[name="features"](%self.1)
  %352 : Tensor = prim::CallMethod[name="forward"](%features, %x)
  %353 : Tensor = prim::CallMethod[name="forward"](%classifier, %352)
  return (%353)

def forward(self,
    x: Tensor) -> Tensor:
  classifier = self.classifier
  features = self.features
  _0 = (classifier).forward((features).forward(x, ), )
  return _0

